TP2: Premier réseau de neurone avec PyTorch
=============

Introduction au sujet
-----

L'objectif de ce sujet est de mettre en place un premier réseau de neurone pour classer des fleurs de la base IRIS.

Le code est à écrire en python3 à la suite des questions dans ce fichier. Vous appuierez soit sur le bouton *run cell*, soit sur les touches *Ctrl-Entrée*, à l’intérieur de la zone de saisie, pour lancer l'exécution de vos commandes. Si la commande est en cours d’exécution une étoile apparaît à côté de la zone de saisie de la commande : In [\*]. Une fois le calcul achevé, l'étoile est remplacée par le numéro du run permettant de retrouver par la suite dans quel ordre ont été lancés chaque bloc.

N'hésitez pas à regarder régulièrement la documentation de ces librairies, des exemples d'utilisation accompagnent généralement l'explication de chaque fonction.

Langage utilisé:
- Python 3: https://docs.python.org/3/

Librairie de math:
- Numpy: https://docs.scipy.org/doc/numpy/reference/
- Scipy: https://docs.scipy.org/doc/scipy/reference/

Librairie d'affichage de données:
- Matplotilb: https://matplotlib.org/contents.html

Librairie Pytorch:
- PyTorch: https: https://pytorch.org/docs/stable/

Commencez par importer les librairies nécessaires au TP.

In [1]:
# Import Torch
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

# Import numpy et matplotlib
import numpy as np
import matplotlib.pyplot as plt

# Import de scikit-learn
import sklearn as sk
import sklearn.datasets
import sklearn.model_selection


PyTorch: Premier réseau sur la base IRIS
-----

Nous allons dans ce TP étudier la base *IRIS* et tester un réseau entièrement connecté dessus. 

Commencez par charger la base *IRIS* avec sckit-learn. Vous mettrez les descripteurs des fleurs dans une variable `X` et les labels dans une variable `y`.

In [2]:
iris = sklearn.datasets.load_iris()
X = iris.data[:, :4]
y = iris.target.astype(np.int64)

Vérifiez que le code suivant affiche bien des *True*.

In [3]:
print(X.shape == (150,4))
print(y.shape == (150,))

True
True


Séparez l'ensemble d'apprentissage en deux en utilisant la fonction train_test_split de sckit-learn. Un ensemble d'apprentissage *train* et un ensemble de *test*. Vous prendrez 1/3 des images pour le test.

In [4]:
train,test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=1/3)

Vérifiez les dimensions des données produites:

In [5]:
print(2*len(X)/3,'->',train.shape,y_train.shape)
print(len(X)/3,'->',test.shape,y_test.shape)

100.0 -> (100, 4) (100,)
50.0 -> (50, 4) (50,)


Définissez un classifieur *iris_classifier* correspondant à un réseau entièrement connecté de 3 couches cachées de tailles [10,20,10]. Après chaque couche cachée, vous appliquerez une fonction d'activation de type ReLU. N'oubliez pas la dernière couche de sortie. Vous utiliseriez `torch.nn.Sequential` pour faire cette question.

In [6]:
model = torch.nn.Sequential(torch.nn.Linear(4, 10),
torch.nn.ReLU()
,torch.nn.Linear(10, 20),
torch.nn.ReLU(),
torch.nn.Linear(20, 10),
torch.nn.ReLU(),
torch.nn.Linear(10, 3)
)


Définissez un objet `iter_train` permettant de parcourir la base de donnée d'entrainement avec des batchs aléatoires de taille 32. Vous utiliserez les classes `TensorDataset` et `DataLoader` pour cette question.

In [7]:
trainset = torch.utils.data.TensorDataset(torch.Tensor(train),torch.tensor(y_train))
iter_train = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

Définissez un objet `iter_test` permettant de parcourir la base de donnée de test avec des batchs de taille 10 concervant l'ordre d'origine des exemples.

In [8]:
testset = torch.utils.data.TensorDataset(torch.Tensor(test),torch.tensor(y_test))
iter_test = torch.utils.data.DataLoader(testset, batch_size=10, shuffle=False)

Définissez une optimiser de type gradient stochastique initialisé avec un taux d'apprentissage de $10^{-2}$.

In [9]:
optim = torch.optim.SGD(model.parameters(),lr=10**-2)

Définissze un critère de type cross-entropie qui sera utilisé comme fonction de coût optimisant notre réseau.

In [10]:
loss =  torch.nn.CrossEntropyLoss()

Effectuez 100 époques d'apprentissage du classifieur `iris_classifier` avec les données de `iter_train`. Vous utiliserez pour celà un algorithme de gradient stochastique avec une fonction de coût de type cross-entropie.

In [11]:
for epoch in range(100):
    model.train()
    for xb,yb in iter_train :
        pred = model(xb)
        l = loss(pred,yb)
        l.backward()
        optim.step()
        optim.zero_grad()
    if(epoch%10==0):
        print("loss =>", l)

loss => tensor(1.0603, grad_fn=<NllLossBackward0>)
loss => tensor(1.1101, grad_fn=<NllLossBackward0>)
loss => tensor(1.0439, grad_fn=<NllLossBackward0>)
loss => tensor(0.9890, grad_fn=<NllLossBackward0>)
loss => tensor(0.8819, grad_fn=<NllLossBackward0>)
loss => tensor(1.1212, grad_fn=<NllLossBackward0>)
loss => tensor(0.9026, grad_fn=<NllLossBackward0>)
loss => tensor(0.9445, grad_fn=<NllLossBackward0>)
loss => tensor(0.8086, grad_fn=<NllLossBackward0>)
loss => tensor(0.9682, grad_fn=<NllLossBackward0>)


Evaluez les performances du réseau appris à la  question précédente sur les données de test de `iter_test`. Pour faire  cette  question vous calculerez dans une boucle le nombre de fois que l'algorithme s'est trompé sur la base de test. Pensez à désactiver le calcul des gradients sur la base de test afin de pas perturbé avec des données de tests de nouveaux apprentissages de votre réseau.

In [12]:
sumPour = 0
i=0
for xb,yb in iter_test :
        pred = model(xb)
        sumPour+=(torch.sum(torch.argmax(pred,axis=1) == yb)/xb.shape[0])*100
        i+=1
print((sumPour/i).item())

80.0


Utilisez tensorboard pour visualiser le graphe correspondant à votre réseau et les différentes courbes correspondant à l'apprentissage de ce dernier.

Vous pouvez pour cela:
- Soit installez le plugin tensorboard pour jupyter: pip3 install --user jupyter-tensorboard

Puis vous suivez les informations d'écrit sur la page: https://github.com/lspvic/jupyter_tensorboard
- Soit lancer dans un terminal: tensorboard --logdir=.

Puis vous vous connectez à http://localhost:6006

In [13]:
writer = SummaryWriter()
writer.add_graph(model,torch.Tensor(train))
writer.close()

PyTorch: Définition d'un réseau couche par couche
-----

Nous allons dans cette partie redéfinir le réseau couche par couche.

Définissez une classe `Net` définissant le réseau précédant sans utiliser de `torch.nn.Sequential`. 

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 10)
        self.fc2 = nn.Linear(10,20)
        self.fc3 = nn.Linear(20,10)
        self.fc4 = nn.Linear(10,3)
    def forward(self, x):
        out = nn.functional.relu(self.fc1(x))
        out2 = nn.functional.relu(self.fc2(out))
        out3 = nn.functional.relu(self.fc3(out2))
        return out3


Apprenez ce réseau sur les données d'apprentissage de la base IRIS avec un algorithme de descende de gradient de type AdaGrad dont le taux d'apprentissage est de $10^{-2}$ avec une fonction de coût de type cross-entropie.

In [15]:
model2 = Net()
optim2 = torch.optim.Adagrad(model2.parameters(),lr=10**-2)
loss2 = torch.nn.CrossEntropyLoss()

for epoch in range(200):
    model2.train()
    for xb,yb in iter_train :
        pred2 = model2(xb)
        l2 = loss2(pred2,yb)
        l2.backward()
        optim2.step()
        optim2.zero_grad()
    if(epoch%100==0):
        print("loss =>", l2)

loss => tensor(2.2100, grad_fn=<NllLossBackward0>)
loss => tensor(0.8955, grad_fn=<NllLossBackward0>)


In [16]:
print(l2)

tensor(0.0915, grad_fn=<NllLossBackward0>)


Testez le réseau que vous venez d'apprendre sur la base de test.

In [17]:
sumPour = 0
i=0
for xb,yb in iter_test :
        pred2 = model2(xb)
        sumPour+=(torch.sum(torch.argmax(pred2,axis=1) == yb)/xb.shape[0])*100
        i+=1
print("Accuracy sur test => " ,(sumPour/i).item(), "%")

Accuracy sur test =>  66.0 %


Sauvegardez le modèle que vous venez d'apprendre dans un fichier.

In [18]:
torch.save(model2.state_dict(),"./modele")

Chargez le réseau que vous venez de sauvegarder et vérifier que les performances sur la base de test n'ont pas changé.

In [19]:
model2.load_state_dict(torch.load('./modele'))
sumPour = 0
i=0
for xb,yb in iter_test :
        pred2 = model2(xb)
        sumPour+=(torch.sum(torch.argmax(pred2,axis=1) == yb)/xb.shape[0])*100
        i+=1
print("Accuracy sur test => " ,(sumPour/i).item(), "%")

Accuracy sur test =>  66.0 %


Comparez les graphes des deux méthodes dans tensorboard. Retrouvez-vous les même éléments ? Qu'est ce qui diffère entre les deux versions ? 

In [20]:
writer = SummaryWriter()
writer.add_graph(model2,torch.Tensor(train))
writer.close()

Comparaison avec un SVM 
----

En utilisant la librairie sckit-learn et le cours de Machine learning du premier semestre, trouver les performances du meilleur SVM sur ces données et comparer les performances avec celle du réseau de neurone.

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import *

parameters={'kernel':['poly','linear','rbf'], 'C':[1,10], 'degree':[2,5,7]}
clf = GridSearchCV(SVC(), parameters)
clf.fit(train,y_train)
print(clf.best_params_,clf.best_score_)
y_pred = clf.predict(test)
print(sklearn.metrics.accuracy_score(y_pred,y_test))



{'C': 1, 'degree': 2, 'kernel': 'poly'} 0.9700000000000001
0.98
